# Limit Function Memory Consumption in Python

The idea is to have a function decorator that tracks the memory consumption of a function and stops the function in case a limit is reached.

The function is run in other process 

Inspired on stackoverflow answer: https://stackoverflow.com/a/10117657

In [1]:
import functools
import multiprocessing 
import os
import psutil
import time

## Required Classes

In [2]:
class MemoryLimiter(multiprocessing.Process):
    def __init__(self, target_function, *args, **kwargs):
        super(MemoryLimiter, self).__init__()
        self.target_function = target_function
        self.args = args
        self.kwargs = kwargs
        self.results = None
        
        self.daemon = True
        self.__has_shutdown = False

    def run(self):
        """ Overloads the threading.Thread.run """

        print(f'Calling {self.target_function.__name__} function...')

        # Start the library Call
        self.results = self.target_function(*self.args, **self.kwargs)

        print(f'{self.target_function.__name__} call complete')
        
        # self.stop()
        self.close()

        self.__has_shutdown = True

    def stop(self):
        self.terminate()

    @property
    def is_running(self):
        return self.is_alive()

    def is_shutdown(self):
        return self.__has_shutdown

## Testing

In [3]:
def memory_hog(set_size, num_dict_entries, delay_per_entry=0):
    """ This function creates a dictionary with 'num_dict_entries' of a set 
    of size 'set_size'
    """
    large_dictionary = {}

    for i in range(num_dict_entries):
        time.sleep(delay_per_entry)
        large_dictionary[f'entry_{i}']=set(range(set_size))

    return len(large_dictionary)

In [4]:
monitor_process = MemoryLimiter(memory_hog, 10000, 5000)
monitor_process.start()

print(monitor_process.is_alive())
print(monitor_process.pid)

True
1516
Calling memory_hog function...
memory_hog call complete


In [9]:
print(monitor_process.is_alive())
print(monitor_process.pid)

False
1516


In [10]:
print(monitor_process.is_alive())
print(monitor_process.pid)

False
1516


How to shares results: https://stackoverflow.com/a/37736655/20966098

In [13]:
monitor_process.results

In [9]:
# memory limit of 2 GB
mem_limit = 2000*(1024.0 ** 2)

monitor_process = MemoryLimiter(memory_hog, 10000, 5000)
monitor_thread.start()

process = psutil.Process(monitor_process.pid)
start_mem = process.memory_full_info().uss


delta_mem, max_memory = 0, 0
while True:
    delta_mem = process.memory_full_info().uss - start_mem
    if delta_mem > max_memory:
        max_memory = delta_mem

    # Check to see if the library call is complete
    if monitor_process.is_shutdown():
        break

print(f'Max Memory Usage in MB: {max_memory / (1024.0 ** 2):.3f}')
print(monitor_process.results) # shouldn't return any values

Calling memory_hog function...
memory_hog call complete
Max Memory Usage in MB: 3987.355
5000


In [10]:
del monitor_thread

### Function Decorator

In [27]:
def mem_monitor(_func=None, *, mem_limit=None):
    
    # convert mem_limit from mb to bytes
    if mem_limit:
        mem_limit = mem_limit*(1024.0 ** 2)

    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            process = psutil.Process(os.getpid())
            start_mem = process.memory_full_info().uss

            monitor_thread = MemoryLimiter(func, *args, **kwargs)
            monitor_thread.start()

            delta_mem, max_memory = 0, 0
            while True:
                delta_mem = process.memory_full_info().uss - start_mem
                if delta_mem > max_memory:
                    max_memory = delta_mem
                    
                if mem_limit and max_memory > mem_limit:
                    print(f'Max memory limit hit: {max_memory / (1024.0 ** 2):.3f} > {mem_limit / (1024.0 ** 2):.3f}')
                    monitor_thread.stop()

                # Check to see if the library call is complete
                if monitor_thread.is_shutdown():
                    break

            print(f'Max Memory Usage in MB: {max_memory / (1024.0 ** 2):.3f}')
            return monitor_thread.results

        return wrapper
    
    if _func is None:
        return decorator
    else:
        return decorator(_func)

@mem_monitor #(mem_limit=2000)
def memory_hog(set_size, num_dict_entries, delay_per_entry=0):
    """ This function creates a dictionary with 'num_dict_entries' of a set 
    of size 'set_size'
    """
    large_dictionary = {}

    for i in range(num_dict_entries):
        time.sleep(delay_per_entry)
        large_dictionary[f'entry_{i}']=set(range(set_size))

    return len(large_dictionary)

memory_hog(10000, 5000)

Calling memory_hog function...
memory_hog call complete
Max Memory Usage in MB: 2361.004


5000

## TODO:

### add controller based on the percentage used memory

In [15]:
psutil.virtual_memory() #.total/(1024.0 ** 3)

svmem(total=16670547968, available=14476906496, percent=13.2, used=1846001664, free=586092544, active=2627522560, inactive=11724406784, buffers=929017856, cached=13309435904, shared=6987776, slab=1397075968)

In [74]:
psutil.virtual_memory().percent > 85

False